# **Keras custom series-Model subclassing 搭建模型**  
使用Keras搭建model除了之前提到的`Sequence`與`Model`方法，還可以使用`Model subclassing`方式搭建。  
這其實就是將所有流程包裝成一個Model類，但是又可以有更高的靈活度(自定義訓練循環同時且享有`fit()`、`evaluation()`等等的method)。

In [1]:
#載入所需lib
import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.2.0


## **最基本的用法**  
有點類似custom layer，在`__init__`中宣告用到的layer，在`call`中處理過程。

In [2]:
class MNIST(tf.keras.Model):
    def __init__(self):
        super(MNIST, self).__init__()
        self.conv_1 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu')
        self.max_pool_1 = tf.keras.layers.MaxPooling2D()
        self.conv_2 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu')
        self.max_pool_2 = tf.keras.layers.MaxPooling2D()
        self.flatten = tf.keras.layers.Flatten()
        self.drop = tf.keras.layers.Dropout(0.5)
        self.out = tf.keras.layers.Dense(10, activation='softmax')
    
    def call(self, inupts):
        x = self.conv_1(inupts)
        x = self.max_pool_1(x)
        x = self.conv_2(x)
        x = self.max_pool_2(x)
        x = self.flatten(x)
        x = self.drop(x)
        return self.out(x)

這樣就定義好了一個model，可以使用`fit()`來訓練。

In [3]:
mnist = MNIST()

#download MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255
y_train = y_train.astype('float32')
x_test = x_test.astype('float32') / 255
y_test = y_test.astype('float32')
x_train = np.expand_dims(x_train, -1)
y_train = np.expand_dims(y_train, -1)
x_test = np.expand_dims(x_test, -1)
y_test = np.expand_dims(y_test, -1)

In [4]:
mnist.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['sparse_categorical_accuracy']
)
history = mnist.fit(x_train, y_train, batch_size=128, epochs=10, validation_split=0.1)

Epoch 1/10
422/422 [==============================] - 2s 4ms/step - loss: 0.3638 - sparse_categorical_accuracy: 0.8916 - val_loss: 0.0863 - val_sparse_categorical_accuracy: 0.9767
Epoch 2/10
422/422 [==============================] - 2s 4ms/step - loss: 0.1126 - sparse_categorical_accuracy: 0.9660 - val_loss: 0.0587 - val_sparse_categorical_accuracy: 0.9847
Epoch 3/10
422/422 [==============================] - 2s 4ms/step - loss: 0.0847 - sparse_categorical_accuracy: 0.9739 - val_loss: 0.0483 - val_sparse_categorical_accuracy: 0.9858
Epoch 4/10
422/422 [==============================] - 2s 4ms/step - loss: 0.0709 - sparse_categorical_accuracy: 0.9786 - val_loss: 0.0405 - val_sparse_categorical_accuracy: 0.9897
Epoch 5/10
422/422 [==============================] - 2s 4ms/step - loss: 0.0628 - sparse_categorical_accuracy: 0.9810 - val_loss: 0.0406 - val_sparse_categorical_accuracy: 0.9877
Epoch 6/10
422/422 [==============================] - 2s 4ms/step - loss: 0.0573 - sparse_categorica

In [5]:
score = mnist.evaluate(x_test, y_test)
print('test loss:{:03.4f}'.format(score[0]))
print('test accuracy:{:3.4f}%'.format(score[1] * 100))

313/313 [==============================] - 1s 2ms/step - loss: 0.0282 - sparse_categorical_accuracy: 0.9906
test loss:0.0282
test accuracy:99.0600%


## **總結**
`Model class`最主要的另一個用途是自定義循環，修改`train_step`，將在下一篇演示。  
這幾種搭建model方式，個人還是偏好使用`Functional API`串接，使用上感覺比較直覺。